In [1]:
#importing the necessary libraries
import tensorflow as tf
import gc
from tensorflow import keras
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from datetime import datetime 
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [3]:
#loading the dataset
file = tf.keras.utils
df = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')

In [4]:
#checking the dataset
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [5]:
#normalizing the data
cleaned_df=df.copy()
scaler = StandardScaler() 
cleaned_df['amount_scaled'] = scaler.fit_transform(cleaned_df['Amount'].values.reshape(-1,1))

# Since The `Amount' column covers a huge range. Lets Convert into to a log-space.
cleaned_df['amount_log'] = np.log(cleaned_df.Amount + 0.0001)

In [6]:
#normalizing the data
cleaned_df=df.copy()
scaler = StandardScaler() 
cleaned_df['amount_scaled'] = scaler.fit_transform(cleaned_df['Amount'].values.reshape(-1,1))

# Since The `Amount' column covers a huge range. Lets Convert into to a log-space.
cleaned_df['amount_log'] = np.log(cleaned_df.Amount + 0.0001)

In [7]:
# Use a utility from sklearn to split and shuffle your dataset.
train_df, test_df = train_test_split(cleaned_df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Class'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('Class'))
test_labels = np.array(test_df.pop('Class'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

In [8]:
#Normalize the input features using the sklearn StandardScaler. 
#This will set the mean to 0 and standard deviation to 1.
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)

Training labels shape: (182276,)
Validation labels shape: (45569,)
Test labels shape: (56962,)
Training features shape: (182276, 32)
Validation features shape: (45569, 32)
Test features shape: (56962, 32)


In [9]:
pd.DataFrame(train_features)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,1.236718,-0.100907,0.749426,-0.485797,0.208587,1.638131,-0.356688,1.302398,-0.359302,-0.373416,...,-0.006770,-0.879958,-0.163107,1.338847,-1.064128,-0.616752,-0.603965,-0.353804,-0.353804,-1.391681
1,-0.291645,-0.251015,0.422015,0.772341,-0.577481,0.367060,-0.156252,0.579886,0.082544,-0.475772,...,-0.150428,-0.319794,-0.873122,-0.346188,0.496831,0.103976,0.327547,-0.276695,-0.276695,0.034025
2,-0.108223,-0.706188,0.122064,0.124137,-1.417400,1.534311,3.080315,-0.297396,1.320339,-0.164214,...,0.635660,-0.437190,1.250225,1.407727,1.392535,-0.353257,-0.157342,-0.053569,-0.053569,0.663045
3,1.334844,-0.059418,0.310358,0.682948,-0.078783,0.124215,-0.396668,0.414617,-0.106260,0.369626,...,1.875877,-0.306635,0.224106,-0.999841,-0.355688,0.660470,0.705284,-0.309748,-0.309748,-0.214879
4,0.520894,0.064896,0.630918,-0.340502,-0.534246,0.917465,-0.149783,0.748323,0.029155,-0.376263,...,-0.958824,0.048979,0.235179,-0.748112,0.247146,0.574658,0.219706,-0.348607,-0.348607,-0.999395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182271,1.257531,1.107854,-0.375561,-0.872697,-0.508129,-0.314588,-0.383960,-0.729414,-0.091080,-0.048026,...,1.021775,0.143748,0.658239,-0.120342,-0.224359,0.072921,-0.018013,-0.276858,-0.276858,0.033072
182272,1.317085,-0.356640,0.587432,-1.986638,-0.634796,0.124350,-0.762010,1.430378,0.154644,-0.597329,...,2.313749,-0.183653,-0.758454,-2.193024,0.193759,-0.246838,0.206167,0.454239,0.454239,1.130308
182273,0.418724,-0.243512,0.129938,-0.684502,-1.376689,-2.100095,0.715584,2.074900,-2.754355,-1.178132,...,-0.860198,0.094891,0.083663,1.975629,1.441020,1.443412,0.086306,2.171434,2.171434,1.671084
182274,0.953928,1.000859,-0.516987,-1.629430,-0.346854,0.335214,-0.295993,0.327944,-0.300218,-0.909226,...,-0.065460,-0.289160,0.196465,0.937874,-0.202489,-0.150302,-0.168234,0.222912,0.222912,0.970720


In [10]:
#defining the model
RF = RandomForestClassifier()

In [17]:
RF = RandomForestClassifier(n_estimators=50,  # number of trees in the forest
                             criterion='gini',  # quality of a split, can also be 'entropy'
                             max_depth=5,  # maximum depth of the tree, None means no limit
                             min_samples_split=2,  # minimum number of samples required to split a node
                             min_samples_leaf=1,  # minimum number of samples required at a leaf node
                             max_features='auto',  # number of features to consider for best split, can be int, float, 'auto', 'sqrt', 'log2' or None
                             bootstrap=True,  # whether bootstrap samples are used when building trees
                             oob_score=False,  # whether to use out-of-bag samples for a generalization error estimate
                             random_state=50,  # seed for random number generator
                             verbose=0,  # control the verbosity of the output
                             class_weight=None)  # weights associated with classes, can be 'balanced', 'balanced_subsample' or a dict

In [18]:
#oversampling on training data
print('Original dataset shape %s' % Counter(train_labels))

ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(train_df,train_labels)

print('Resampled dataset shape %s' % Counter(y_train_ros))

Original dataset shape Counter({0: 181960, 1: 316})
Resampled dataset shape Counter({0: 181960, 1: 181960})


In [19]:
#oversampling on testing data
print('Original dataset shape %s' % Counter(test_labels))

ros = RandomOverSampler(random_state=42)
X_test_ros, y_test_ros = ros.fit_resample(test_df,test_labels)

print('Resampled dataset shape %s' % Counter(y_test_ros))

Original dataset shape Counter({0: 56857, 1: 105})
Resampled dataset shape Counter({0: 56857, 1: 56857})


In [20]:
k_fold= KFold(n_splits=5, shuffle=True, random_state=50)

In [21]:
cv_scores = cross_val_score(RF,X_train_ros,y_train_ros , cv=k_fold, scoring='accuracy')

In [22]:
np.mean(cv_scores)

0.9531462958892064

In [23]:
cv_scores

array([0.95184381, 0.95422071, 0.95362992, 0.9505798 , 0.95545724])

In [24]:
std_score = np.std(cv_scores)

In [25]:
std_score

0.0017327496993952707